# Analyze Elon's timeline

#### Load Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import altair as alt
import altair_stiles as altstiles

In [3]:
alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("grid")

ThemeRegistry.enable('grid')

In [4]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

---

## Get timeline data

In [7]:
src = pd.read_csv(
    "data/processed/elonmusk_timeline_full.csv",
    dtype={"id": str, "conversation_id": str, "in_reply_to_user_id": str},
)

In [21]:
src["free_speech"] = src["text"].str.lower().str.contains("free speech")

In [22]:
src

,source,conversation_id,id,in_reply_to_user_id,text,idnum,retweet_count,reply_count,like_count,quote_count,date,id_num,free_speech
0,Twitter for iPhone,1520655019866959873,1520657179908448258,5943622,@pmarca Haha he says “Bitcoin” so many times,NaN,847,1040,11082,86,2022-04-29,1520657179908448258,False
1,Twitter for iPhone,1520650036865949696,1520650036865949696,NaN,Since I’ve been asked a lot:\n\nBuy stock in s...,NaN,50485,19792,459486,4455,2022-04-29,1520650036865949696,False
2,Twitter for iPhone,1520633450562134016,1520645386427195392,272343476,@vallioo @pmarca Munger is correct,NaN,140,177,2212,12,2022-04-29,1520645386427195392,False
3,Twitter for iPhone,1520633450562134016,1520644460710834176,1337433908074467328,@TheCaptain_Nemo @pmarca Major problem. Passiv...,NaN,151,109,2058,12,2022-04-29,1520644460710834176,False
4,Twitter for iPhone,1520633450562134016,1520643441994387456,3639301,@pegobry @pmarca Exactly. Right before he died...,NaN,501,281,4625,77,2022-04-29,1520643441994387456,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17585,Twitter for iPhone,142881284019060736,142881284019060736,NaN,That was a total non sequitur btw,1.428813e+17,9,9,31,4,2011-12-03,142881284019060736,False
17586,Twitter for iPhone,142880871391838208,142880871391838208,NaN,"Great Voltaire quote, arguably better than Twa...",1.428809e+17,9,8,33,0,2011-12-03,142880871391838208,False
17587,Twitter Web Client,142188458125963264,142188458125963264,NaN,I made the volume on the Model S http://t.co/w...,1.421885e+17,16,6,122,4,2011-12-01,142188458125963264,False
17588,Twitter Web Client,142179928203460608,142179928203460608,NaN,Went to Iceland on Sat to ride bumper cars on ...,1.421799e+17,21,33,261,4,2011-12-01,142179928203460608,False


In [91]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')
text = 'You are a terrible person. But I love love my frogs. Still you are the worst person. Fun.'

doc = nlp(text)

In [ ]:
text = 'You are a terrible person. But I love love my frogs. Still you are the worst person. Fun.'

doc = nlp(text)

In [93]:
doc._.blob.polarity

-0.13999999999999999

In [94]:
doc._.blob.sentiment

Sentiment(polarity=-0.13999999999999999, subjectivity=0.68)

In [95]:
doc._.blob.subjectivity 

0.68

In [99]:
def sentiment_analysis(tweet):
    def getSubjectivity(text):
        return TextBlob(text).sentiment.subjectivity
    
    def getPolarity(text):
        return TextBlob(text).sentiment.polarity
  
    tweet["subjectivity"] = tweet["tweet"].apply(getSubjectivity)
    tweet ["polarity"] = tweet["tweet"].apply(getPolarity)
    
    def getAnalysis(score):
        if score < 0:
            return "Negative"
        elif score == 0:
            return "Neutral"
        else:
            return "Positive"
        tweet["TextBlob_Analysis"] = tweet["TextBlob_Polarity"].apply(getAnalysis)
    return tweet